# Apriori-Algorithmus
Implementieren Sie den Apriori-Algorithmus zum Finden häufiger Item Sets bei einer gegebenen Menge von Transaktionen. Nutzen Sie das gegebene Grundgerüst und beachten Sie folgende Anforderungen:
- Hinweis: Achten Sie besonders bei der Apriori Kandidatengenerierung darauf, dass Sie den Support möglichst effizient und nur von möglichst wenigen Itemsets berechnen.
- Die Ausgabe des Programms ist eine Liste aller Frequent Item Sets zusammen mit ihrem jeweiligen Support. Diese sollen in einer sinnvollen Datenstruktur zurückgegeben werden.
- Sie können Ihr Programm zum Beispiel auf dem Datensatz warenkorb.csv testen.
- Ihre Implementierung können Sie bis zum 09.01.2023 23:59 im Moodle abgeben.
- Sie dürfen die externen Pakete collecions und numpy verwenden. Aus dem Paket collections sollte vor allen Dingen die Klasse *Counter* für Sie hilfreich sein.

## Zusatzaufgabe für 6 ECTS:
**Hinweis**: Bearbeiten Sie diese Aufgaben nur, falls Sie am Praktikum für 6 ECTS-Punkte teilnehmen, jedoch nicht, falls Sie am Praktikum für 3 ECTS-Punkte teilnehmen.
### Anti-Monotone Constraints
- In der Vorlesung haben Sie anti-monotone Constraints zur Einschränkung der relevanten Itemsets kennengelernt.
- Implementieren Sie eine Möglichkeit, mittels des Parameters *prices* ein Dictionary mitzugeben, welches die Preise der einzelnen Items mitgibt.
- Durch den Parameter *max_price* soll dann eine mögliche Preisgrenze gegeben werden. Das heißt, es sollen nur frequent Itemsets zurückgegeben werden, welche nicht mehr als *max_price* kosten. Falls *max_price* den Wert None hat, soll der Constraint des maximalen Preises nicht genutzt werden.
- Nutzen Sie die Anti-Monotonie des Constraints aus, um die Generierung unnötiger Kandidaten zu vermeiden. Es ist NICHT ausreichend, alle frequent Itemsetzs erst zu berechnen um dann ganz zum Schluss die "zu teuren" zu entfernen!

### Konfidenz und Interessantheit
- Der Apriorie Algorithmus liefert Ihnen die Frequent Itemsets und ihren Support. Berechnen Sie aus diesen die Assoziationsregeln, welche eine vorgegebene Konfidenz überschreiten.
- Implementieren Sie die Funktion *interestingness* so, dass sie die Assozationsregeln, welche eine gewisse Konfidenz überschreiten und deren Interessantheit zurückgeben!

## Erinnerung
Bitte vergessen Sie nicht, sich offiziell im Praktikum einzuschreiben! Sollten Sie Fragen dazu haben, wenden Sie sich bitte per E-Mail an stubbemann@cs.uni-kassel.de.

# Daten laden
Diesen Teil bitte nicht verändern.

In [37]:
with open('warenkorb.csv') as f:
    data = []
    for line in f.readlines():
        items = [item.strip() for item in line.split(',')]
        items.sort()
        data.append(items)
data

[['Bier', 'Chips', 'Windeln'],
 ['Chips', 'TV-Zeitschrift'],
 ['Bier', 'Chips', 'TV-Zeitschrift'],
 ['Bier', 'Windeln', 'Zahnpasta'],
 ['Chips', 'Zahnpasta'],
 ['Bier', 'Chips', 'TV-Zeitschrift'],
 ['Bier', 'Windeln'],
 ['Chips', 'TV-Zeitschrift']]

In [38]:
prices = {"Chips": 2.5,
          "Bier": 5,
          "TV-Zeitschrift": 2,
          "Windeln": 1.5,
          "Zahnpasta": 2}

# Implementierung
Implementieren Sie in diesem Teil Ihre Lösung. Die gegebenen Datenstrukturen und Methoden können Sie als Hilfestellung (insbesondere für eine effiziente Lösung) benutzen, müssen Sie aber nicht. Nur die Apriori-Methode sollte so funktionieren wie  gegeben. Sie können natürlich weitere Methoden hinzufügen.

In [105]:
from collections import Counter
import numpy as np
import itertools


def item_count(transactions):
    c = Counter([item for t in transactions for item in t])
    return c.most_common()

def apriori(transactions,
            prices=prices,
            max_price=None,
            minsup=0.01):
    # The parameters prices and max_price are just needed for 6ECTS!
    l = {}
    #1 Element Itemsets aus Transaktions
    l1 = item_count(transactions)
    
    if max_price == None:
        dct = dict((Itemset([x]),y/len(transactions)) for x, y in l1 if y/len(transactions) >= minsup)
    else:
        dctWithoutMaxPrice = dict((Itemset([x]),y/len(transactions)) for x, y in l1 if y/len(transactions) >= minsup)
        dct ={}
        for x,y in dctWithoutMaxPrice.items():
            if prices[x.items[0]] < max_price:
                dct[x] = y
                
    l.update(dct)
    k = 2 
    #candidate Item in Itemsets in keys gespeichert
    currentList = [key for key in dct.keys()]
    dct.clear()

    while len(currentList)!= 0:
        candidate = aprioriKandidatenGenerierung(currentList,minsup,k,max_price)
        dct = returnItemsWithMinSupport(transactions,minsup,k,candidate)
        l.update(dct)
        currentList.clear()
        currentList = [key for key in dct.keys()]
        k += 1
    
    return list(l.items())

def aprioriKandidatenGenerierung(currentList,minsup,k,max_price):
    #alle Element in currentList mit Format String reigekommen
    #speichern Element mit Wrapper Itemset
    #für k>2
    candidate = set()
    candidateTmpSet = set()
    lenCurrentList = len(currentList)
    
    if k == 2:
        for i in currentList:
            elementListSetFirst = set(i.items)
            for j in currentList:
                elementListSetSecond = set(j.items)
                newSet = elementListSetFirst.union(elementListSetSecond)
                newSetList = sorted(newSet)
                if max_price == None:
                    if len(newSetList) != 1:
                        candidate.add(Itemset(newSetList))
                else:
                    price = 0
                    for item in newSetList:
                        price += prices[item]
                    if price < max_price:
                        candidate.add(Itemset(newSetList))

    if k > 2:
        #1.Join
        for i in currentList:
            elementListSetFirst = set(i.items)
            for j in currentList:
                elementListSetSecond = set(j.items)
                if not (i == j):
                    commonSet = elementListSetFirst.intersection(elementListSetSecond)
                    fistKElementSet = set(i.items[0:k-2])
                    if commonSet == fistKElementSet:
                        candidateSet = elementListSetFirst.union(elementListSetSecond)
                        candidateSetList = sorted(candidateSet)
                        if max_price == None:
                            candidateTmpSet.add(Itemset(candidateSetList))
                        else:
                            price = 0
                            for item in candidateSetList:
                                price += prices[item]
                            if price < max_price:
                                candidateTmpSet.add(Itemset(candidateSetList))
                        
        #2. pruning
        currentListWithoutWrapper = [item.items for item in currentList]
        for i in candidateTmpSet:
            itemSet = set(i.items)
            subSet = list(itertools.combinations(itemSet, k))
            
            for j in subSet:
                if j not in currentListWithoutWrapper:
                    break
            candidate.add(i)
            
    return candidate
            

def returnItemsWithMinSupport(transactions,minsup,k,candidate):
    # keys in dictionary basteln und values mit 0 initialisieren
    dct = {item: 0 for item in candidate}
    #jede Element nach k basteln Zähler hochzählen
    for transaction in transactions: 
        transactionSet = set(transaction)
        for element in candidate:
            elementSet = set(element.items)
            if elementSet.issubset(transactionSet):
                dct[element] += 1
                
    dctWithMinsup = {x:y/len(transactions) for (x, y) in dct.items() if y/len(transactions)>=minsup}
    return dctWithMinsup

def interestingness(l, min_conf):
    """
    This function is only needed for 6ECTS!
    The list l contains all frequent itemsets and their support.
    Compute fom all frequent Itemsets X and all their subsets A the interestingness of the rule
    A -> X\A if this rule has at least the confidence min_conf.
    Returns a list of tuples where each tuple has the form (A, X\A , interstingness of A -> X\A)
    """
    # TODO
    pass


class Itemset():
    """ A hashable itemset that can be used as a key in dictionaries or as an element in sets.
    You can change this class how ever you like. """
    
    def __init__(self, items):
        self.items = items
    
    def __str__(self):
        return str(self.items)
    
    def __repr__(self):
        return str(self)
    
    def __eq__(self, other):
        return len(self.items) == len(other.items) and set(self.items).issubset(other.items)
    
    def __hash__(self):
        return hash(str(self))
    
    
    
# Calculate frequent itemsets for the given data-set and print the result.
l = apriori(data, minsup=0.1)
l

[(['Chips'], 0.75),
 (['Bier'], 0.625),
 (['TV-Zeitschrift'], 0.5),
 (['Windeln'], 0.375),
 (['Zahnpasta'], 0.25),
 (['Chips', 'Windeln'], 0.125),
 (['Chips', 'Zahnpasta'], 0.125),
 (['Bier', 'Zahnpasta'], 0.125),
 (['Chips', 'TV-Zeitschrift'], 0.5),
 (['Bier', 'Chips'], 0.375),
 (['Windeln', 'Zahnpasta'], 0.125),
 (['Bier', 'Windeln'], 0.375),
 (['Bier', 'TV-Zeitschrift'], 0.25),
 (['Bier', 'Chips', 'TV-Zeitschrift'], 0.25),
 (['Bier', 'Chips', 'Windeln'], 0.125),
 (['Bier', 'Windeln', 'Zahnpasta'], 0.125)]

## Tests für die 6ECTS Aufgaben
Löschen Sie die folgenden Zellen, wenn Sie nur 3ECTS benötigen!

In [102]:
l = apriori(data, minsup=0.2, max_price=7)
l

{['Chips']: 0.75,
 ['Bier']: 0.625,
 ['TV-Zeitschrift']: 0.5,
 ['Windeln']: 0.375,
 ['Zahnpasta']: 0.25,
 ['Chips', 'TV-Zeitschrift']: 0.5,
 ['Bier', 'Windeln']: 0.375}

In [103]:
l = apriori(data, minsup=0.01)
i = interestingness(l, min_conf=0.3)
i

In [104]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:85% !important; }</style>"))